In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from dateutil.parser import parse
from datetime import datetime

In [2]:
respiratory_data = pd.read_csv('respiratory_data.csv')

In [3]:
enrollment= pd.read_csv('enrollment_countries_respiratory.csv')

In [4]:
enrollment['NCT Number'] = enrollment['NCT Number'].str[3:].astype('int')

In [5]:
respiratory_data['NCT Number'] = respiratory_data['NCT Number'].str[3:].astype('int')

In [6]:
my_df= respiratory_data.set_index('NCT Number').join(enrollment.set_index('NCT Number'))

In [7]:
my_df['Actual'] = my_df['Actual'].replace('Not Provided',np.NaN)

In [8]:
my_df['Actual'] = my_df['Actual'].astype('float')

In [9]:
my_df['Estimated'] = np.where(my_df['Estimated'] == 'Same as current',my_df['Actual'], my_df['Estimated'])

In [10]:
my_df['Estimated']= my_df['Estimated'].replace('Not Provided',np.NaN)

In [11]:
my_df['Estimated'] = my_df['Estimated'].astype('float')

In [12]:
my_df['EFraction']= my_df['Actual']/my_df['Estimated']

In [13]:
status_dict = {"Terminated" : 0 , "Completed" : 1}

In [14]:
my_df =my_df.replace({"Status": status_dict})

In [15]:
my_df["Phases"].unique()

array(['Not Applicable', 'Phase 4', 'Phase 2', 'Phase 3', 'Phase 1',
       'Phase 2|Phase 3', 'Phase 1|Phase 2', 'Early Phase 1', nan],
      dtype=object)

In [16]:
my_df['Countries']= my_df['Countries'].replace('Not Provided',np.NaN)

In [17]:
my_df["Countries"]= my_df["Countries"].str.split(',')

In [18]:
my_df["Intervention"]= my_df["Interventions"].str.split(': ').str[0]

In [19]:
my_df['Intervention'].unique()

array(['Other', 'Device', 'Drug', 'Behavioral', 'Diagnostic Test',
       'Procedure', 'Biological', 'Combination Product',
       'Dietary Supplement', 'Radiation', 'Genetic'], dtype=object)

In [20]:
intervention_dict = {'Drug': 0, 'Behavioral' : 1, 'Radiation': 2, 'Procedure':3,
       'Dietary Supplement':4, 'Biological':5, 'Diagnostic Test':6, 'Device':7,
       'Genetic':8, 'Combination Product':9, 'Other': 10}

In [21]:
my_df =my_df.replace({"Interventions": intervention_dict})

In [22]:
my_df['Disease']= 'Respiratory'

In [23]:
my_df['Start Date'] = pd.to_datetime(my_df['Start Date'], errors='coerce').dt.to_period('M')

In [24]:
my_df['Completion Date'] = pd.to_datetime(my_df['Completion Date'], errors='coerce').dt.to_period('M')

In [25]:
my_df['Month']= 12 * (my_df['Completion Date'].dt.year - my_df['Start Date'].dt.year) + (my_df['Completion Date'].dt.month - my_df['Start Date'].dt.month)

In [26]:
my_df

,Rank,Title,Status,Study Results,Conditions,Interventions,Outcome Measures,Gender,Age,Phases,...,Completion Date,Locations,URL,Actual,Estimated,Countries,EFraction,Intervention,Disease,Month
NCT Number,,,,,,,,,,,,,,,,,,,,,
2898129,1,Evaluating the Relationship Between Environmen...,1,No Results Available,Chronic Respiratory Diseases,Other: Questionnaire|Device: Lung function test,FEV1/FVC|FVC|VC|FEF 25-75%|Socio-demographic s...,All,"10 Years to 65 Years (Child, Adult, Older Ad...",Not Applicable,...,2017-07,"Pham Ngoc Thach University of medicine, Ho Chi...",https://ClinicalTrials.gov/show/NCT02898129,1500.0,1500.0,[Vietnam],1.000000,Other,Respiratory,10
3333850,2,Technology Assisted Physical Activity Among Ho...,1,No Results Available,Respiratory Disease,Device: Visual feedback of physical activity|D...,Time spent out of bed during hospitalisation|T...,All,"18 Years and older (Adult, Older Adult)",Not Applicable,...,2018-06,"Bispebjerg Hospital, Department of Physical an...",https://ClinicalTrials.gov/show/NCT03333850,96.0,108.0,[Denmark],0.888889,Device,Respiratory,7
3334916,3,A Clinical Trial to Evaluate the Efficacy and ...,1,No Results Available,Respiratory Disease,Drug: YMC026|Drug: Placebo,Change from Baseline BSS at Day 6|Change from ...,All,"15 Years to 74 Years (Child, Adult, Older Ad...",Phase 4,...,2018-10,"The Catholic University of Korea, Seoul St.Mar...",https://ClinicalTrials.gov/show/NCT03334916,223.0,216.0,"[Korea, Republic of]",1.032407,Drug,Respiratory,12
3595488,4,Dupilumab for Aspirin-exacerbated Respiratory ...,1,No Results Available,Aspirin-exacerbated Respiratory Disease,Drug: Dupilumab,SNOT 22 Score|UPSIT|Lund Mackay score|ACT scor...,All,"18 Years and older (Adult, Older Adult)",Phase 2,...,2019-12,Rochester Regional Health - Allergy/Immunology...,https://ClinicalTrials.gov/show/NCT03595488,11.0,10.0,[United States],1.100000,Drug,Respiratory,15
4623762,5,"Yoga On Dyspnea, Sleep And Fatıgue In Chronıc ...",1,No Results Available,Chronic Respiratory Disease,Other: Yoga,Chronic Obstructive Pulmonary Disease and Asth...,All,18 Years to 55 Years (Adult),Not Applicable,...,2020-07,"Fırat university, Erzurum, Center, Turkey|Saba...",https://ClinicalTrials.gov/show/NCT04623762,60.0,60.0,[Turkey],1.000000,Other,Respiratory,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405310,9996,Convalescent Plasma of Covid-19 to Treat SARS-...,1,No Results Available,SARS Pneumonia,Biological: Convalescent Plasma of patients wi...,Death|Lenth of stay ICU|Days of Mechanical Ven...,All,"18 Years to 70 Years (Adult, Older Adult)",Phase 2,...,2020-12,"Hospital Nava de Alta Especialidad, Mexico Cit...",https://ClinicalTrials.gov/show/NCT04405310,42.0,80.0,[Mexico],0.525000,Biological,Respiratory,7
1187147,9997,Effects of Green Tea Extract (GTE) on Systemic...,1,No Results Available,Asthma,Dietary Supplement: green tea extract,Serum Inflammatory markers|Respiratory symptoms,All,"18 Years to 70 Years (Adult, Older Adult)",Not Applicable,...,2010-07,"University of Hong Kong Queen Mary Hospital, H...",https://ClinicalTrials.gov/show/NCT01187147,30.0,30.0,[Hong Kong],1.000000,Dietary Supplement,Respiratory,30
3824366,9998,Same-session MR-only Simulation and Treatment ...,1,No Results Available,Malignancy|Metastasis|Hemoptysis|Gastrointesti...,Device: Volumetric MR imaging|Radiation: Radia...,Feasibility of same-session MRI-only simulatio...,All,"18 Years and older (Adult, Older Adult)",Not Applicable,...,2022-01,"Washington University School of Medicine, Sain...",https://ClinicalTrials.gov/show/NCT03824366,20.0,20.0,[United States],1.000000,Device,Respiratory,33


In [27]:
my_df.to_csv('respiratory_data_cleaned.csv',index=False)